# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). 

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**
26. **META: Meta-model**
27. **VAE: Variational Autoencoder**

In [3]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pythresh is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.knn import KNN
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP
from pythresh.thresholds.meta import META
from pythresh.thresholds.vae import VAE

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [4]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 
              'KARCH', 'OCSVM', 'CLUST', 'DECOMP', 'META', 'VAE']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP(),
        'Meta-model (META)': META(),
        'Variational Autoencoder (VAE)': VAE()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        #print('{thres_name} ROC:{roc}, precision @ rank n:{prn}, '
        #      'execution time: {duration}s'.format(
        #thres_name=thres_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...

... Processing cardio.mat ...

... Processing glass.mat ...

... Processing ionosphere.mat ...

... Processing letter.mat ...

... Processing lympho.mat ...

... Processing mnist.mat ...

... Processing musk.mat ...

... Processing optdigits.mat ...

... Processing pendigits.mat ...

... Processing pima.mat ...

... Processing satellite.mat ...

... Processing satimage-2.mat ...

... Processing vertebral.mat ...

... Processing vowels.mat ...

... Processing wbc.mat ...


In [5]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE
0,arrhythmia,452,274,14.6018,0.8082,0.6797,0.7374,0.931,0.6678,0.7212,...,0.6738,0.6707,0.7428,0.6751,0.6687,0.8448,0.6841,0.7111,2.764,3.1892
0,cardio,1831,21,9.6122,0.6478,0.7479,0.7671,1.4155,0.6461,0.8407,...,0.65,0.6447,0.9626,0.6662,0.6878,1.0696,0.869,0.9454,2.8226,10.2124
0,glass,214,9,4.2056,0.4738,0.5026,0.506,0.7052,0.4738,0.4912,...,0.4771,0.4752,0.5107,0.4791,0.4756,0.6331,0.479,0.4935,2.5181,1.5853
0,ionosphere,351,33,35.8974,0.5108,0.5526,0.5323,0.739,0.5082,0.5466,...,0.5228,0.5148,0.5793,0.5209,0.5198,0.6996,0.529,0.5466,2.5582,2.4814
0,letter,1600,32,6.25,0.6739,0.706,0.7443,1.3787,0.6633,0.8563,...,0.6632,0.6583,0.9306,0.6639,0.6944,1.0778,0.7703,0.8981,2.8713,9.0933
0,lympho,148,18,4.0541,0.4728,0.5023,0.5042,0.7532,0.4723,0.483,...,0.4733,0.4703,0.4983,0.472,0.4727,0.6262,0.4757,0.4835,2.4984,1.4075
0,mnist,7603,100,9.2069,2.01,2.0884,4.1424,4.8936,2.0099,4.014,...,2.076,2.0081,3.3774,2.0534,2.8279,5.2736,6.3857,5.4389,4.615,40.9261
0,musk,3062,166,3.1679,1.44,1.4166,1.7668,2.2912,1.4422,1.9256,...,1.41,1.4014,1.9399,1.4138,1.5365,2.0554,2.1073,2.1054,3.6783,16.9389
0,optdigits,5216,64,2.8758,1.3719,1.3979,2.3442,3.0815,1.3446,2.4568,...,1.3551,1.3442,2.2603,1.3671,1.7113,2.8676,3.2808,3.0596,3.84,27.478
0,pendigits,6870,16,2.2707,1.1433,1.1687,2.8502,3.7684,1.1088,2.7674,...,1.1211,1.1107,2.3302,1.1612,1.7527,3.6001,5.8073,3.9682,3.7415,36.4689


In [6]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE
0,arrhythmia,452,274,14.6018,0.8163,0.8578,0.8455,0.8133,0.835,0.8336,...,0.8375,0.8548,0.8576,0.8147,0.8403,0.8172,0.8466,0.8345,0.8597,0.838
0,cardio,1831,21,9.6122,0.9345,0.9342,0.9334,0.9114,0.9407,0.9188,...,0.9038,0.942,0.9304,0.9261,0.9034,0.9179,0.9274,0.901,0.9221,0.9148
0,glass,214,9,4.2056,0.6667,0.6247,0.6296,0.6642,0.6444,0.6938,...,0.6469,0.7037,0.6494,0.6346,0.6519,0.642,0.6296,0.6198,0.6395,0.6864
0,ionosphere,351,33,35.8974,0.7844,0.7769,0.7819,0.7899,0.786,0.7842,...,0.7757,0.768,0.7817,0.768,0.7799,0.7757,0.7888,0.7723,0.7886,0.776
0,letter,1600,32,6.25,0.5906,0.6035,0.5987,0.5904,0.5806,0.6071,...,0.5677,0.6585,0.5961,0.6206,0.594,0.6753,0.627,0.6129,0.6883,0.6159
0,lympho,148,18,4.0541,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.9942,1.0,1.0,0.9942,0.9942,0.9942,1.0,1.0,1.0
0,mnist,7603,100,9.2069,0.8147,0.8011,0.8123,0.7992,0.7736,0.7773,...,0.8196,0.7826,0.8127,0.7941,0.7692,0.7976,0.8062,0.7882,0.8005,0.8099
0,musk,3062,166,3.1679,0.9997,0.9997,0.9998,0.9995,0.9973,0.9999,...,0.9996,0.9997,0.9997,0.9988,1.0,1.0,1.0,1.0,1.0,0.9906
0,optdigits,5216,64,2.8758,0.6554,0.7185,0.7661,0.7682,0.68,0.6271,...,0.6948,0.7126,0.7197,0.7483,0.6892,0.7659,0.7221,0.7606,0.6923,0.6995
0,pendigits,6870,16,2.2707,0.9413,0.9294,0.948,0.9468,0.9421,0.9236,...,0.9349,0.9256,0.9442,0.9437,0.959,0.9222,0.9528,0.943,0.9452,0.919


In [7]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META,VAE
0,arrhythmia,452,274,14.6018,0.4643,0.6071,0.5,0.5714,0.5714,0.6429,...,0.5714,0.6071,0.5357,0.5357,0.5357,0.5,0.5714,0.5357,0.5357,0.5
0,cardio,1831,21,9.6122,0.5429,0.5286,0.4857,0.5143,0.5714,0.4857,...,0.4714,0.5857,0.5,0.5286,0.4429,0.4857,0.4857,0.4,0.5143,0.4571
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5652,0.5435,0.587,0.5217,0.5217,0.5652,...,0.5217,0.5217,0.587,0.5,0.5217,0.5217,0.5435,0.5435,0.587,0.5217
0,letter,1600,32,6.25,0.0732,0.0732,0.0976,0.0732,0.0976,0.0488,...,0.0732,0.0976,0.0488,0.0488,0.0488,0.0976,0.122,0.0976,0.0732,0.0732
0,lympho,148,18,4.0541,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.6667,1.0,1.0,0.6667,0.6667,0.6667,1.0,1.0,1.0
0,mnist,7603,100,9.2069,0.3111,0.3111,0.3148,0.2778,0.2704,0.2444,...,0.3556,0.2556,0.3185,0.2852,0.2407,0.2815,0.3111,0.2704,0.2815,0.3074
0,musk,3062,166,3.1679,0.9756,0.9512,0.9512,0.9512,0.8293,0.9756,...,0.9024,0.9756,0.9512,0.9024,1.0,1.0,1.0,1.0,0.975,0.6829
0,optdigits,5216,64,2.8758,0.0154,0.0154,0.0615,0.0615,0.0308,0.0154,...,0.0154,0.0154,0.0462,0.0154,0.0154,0.0308,0.0154,0.0462,0.0154,0.0154
0,pendigits,6870,16,2.2707,0.2742,0.2258,0.2742,0.2258,0.2419,0.2419,...,0.2581,0.2097,0.2581,0.2419,0.3387,0.2419,0.2258,0.2742,0.3226,0.2097
